# Fase 6: Deployment

Esse notebook será usado para salvar o melhor modelo que encontramos na etapa de treinamento e apresentar o código de deployment. Faremos o deploy utilizando a ferramenta BentoML, e disponibilizaremos o modelo como RestAPI via AWS Ec2 utilizando uma estratégia de containers com Docker.

### Grupo
 - Nilo Bemfica (nbmcd)
 - Pedro Didier (pdm)
 - Pedro Tenório (ptl)

# Imports

In [1]:
import os
import bentoml
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

c:\Users\Pedro\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Fazendo os ajustes na base para ficar igual ao que tivemos na etapa 4.

In [2]:
def time_to_int(date_str):
    date = pd.to_datetime(date_str)

    total = int(date.strftime('%S'))
    total += int(date.strftime('%M')) * 60
    total += int(date.strftime('%H')) * 60 * 60
    total += (int(date.strftime('%j')) - 1) * 60 * 60 * 24
    # 2002 é o ano mais antigo no dataset
    total += (int(date.strftime('%Y')) - 2002) * 60 * 60 * 24 * 365
    return total

In [3]:
seed_value= 42
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
# Carregando base
df = pd.read_csv("homologated_data.csv")
# Corrigindo formatos de dado
df = df.drop(columns=["C_man", "C_api"])
df = df[df["gender"] != 0].reset_index(drop=True)
df["firstDay"] = df["firstDay"].apply(time_to_int)
df["lastDay"] = df["lastDay"].apply(time_to_int)
# Train test split com seed que utilizamos no experimento
X = df.drop(columns=["gender"]).values
y = df["gender"].values
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    random_state=seed_value
)

# Salvando o modelo como Pickle

In [4]:
import pickle

In [5]:
# Selecionando os melhores parametros
clf = RandomForestClassifier(n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_depth=7, bootstrap=True)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_leaf=4, min_samples_split=10)

In [6]:
with open('best_model.pkl', 'wb') as file:
    pickle.dump(clf, file)

# Deploy com BentoML

Vamos carregar o modelo e salvar como um bentoml model, para poder utilizar as funcionalidades do Framework

In [7]:
with open('best_model.pkl', 'rb') as file:
    clf = pickle.load(file)

In [8]:
saved_model = bentoml.sklearn.save_model("rf_gender_classifier", clf)

## Criando um endpoint

Em bentoml, para criar uma API que sirva o modelo, basta criar um 'service' com um endpoint que utilize um 'runner' do modelo que você salvou. Um runner é nada mais que um tipo de worker específico do bentoml, que pode ser spawnado diversas vezes conforme a demanda da aplicação.  

* Para informações mais detalhadas, aqui está a documentação do BentoML: https://docs.bentoml.com/en/latest/index.html

In [9]:
from bentoml.io import NumpyNdarray

gender_clf_runner = bentoml.sklearn.get("rf_gender_classifier:latest").to_runner()

svc = bentoml.Service("gender_classifier", runners=[gender_clf_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    result = gender_clf_runner.predict.run(input_series)
    return result

Aqui, já é possível servir o modelo localmente, podemos rodar no terminal:  

In [10]:
!bentoml serve service:svc
# Rodar no notebook não vai te permirtir rodar mais nenhuma outra célula, então se for testar rode em um terminal com um python env com os requirements

^C


E fazer uma request para testar o endpoint:

In [15]:
import requests

# Linha a ser inferida
to_predict = [list(X_test[0])]

# Fazendo um post para nosso endpoint
response = requests.post(
   "http://127.0.0.1:3000/classify",
   headers={"content-type": "application/json"},
   data=str(to_predict),
).text

In [16]:
print(f"Linha prevista como {response}, onde 1 é homem e 2 é mulher.")

Linha prevista como [1], onde 1 é homem e 2 é mulher.


Para criar um docker da aplicação basta definir um arquivo de deployment chamado **bentofile.yaml** da seguinte forma:

```yaml

service: "service:svc"  # Same as the argument passed to `bentoml serve`
labels:
   owner: bentoml-team
   stage: dev
include:
- "*.py"  # A pattern for matching which files to include in the Bento
python:
   packages:  # Additional pip packages required by the Service
   - scikit-learn
models: # The model to be used for building the Bento.
- rf_gender_classifier:latest

E então seguir os seguintes passos no terminal:
 * bentoml build  
**retorna**  
Locking PyPI package versions.  
 BENTOML  
Successfully built Bento(tag="rf_gender_classifier:awln3pbmlcmlonry").


 * bentoml containerize **rf_gender_classifier:<tag_do_container>** # Dada pelo bentoml quando rodamos a linha anterior

 * bentoml list  
**retorna**  
Tag                               Size       Creation Time  
rf_gender_classifier:awln3pbmlcmlonry  78.84 MiB  2023-09-07 16:38:42

Com isso, já temos nosso container da aplicação localmente com Docker. Podemos testar isso com:
 * docker run -p 3000:3000 rf_gender_classifier:awln3pbmlcmlonry

# Deploy no AWS Ec2

Requerimentos para deploy

 - Terraform - Terraform: https://www.terraform.io/downloads
 - AWS CLI - instalado e configurado com permissão para EC2 e ECR: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
 - Docker - Instalação: https://docs.docker.com/install
 - bentoctl instalado (pip install bentoctl)


### Instale o operator necessário  
* bentoctl operator install aws-ec2  

### Inicialize o deployment com o bentoctl
* bentoctl init

### Aqui, você deve configurar seu deploy
 * name: nome-do-endpoint-da-ia  
 * region: us-east-2  
 * operator: aws-ec2 
 * instance_type: t2.micro
 * ami_id: ami-0cf0e376c672104d6
 * enable_gpus: False
 * sem variáveis de ambiente
 * escolha um nome para o deployment_config

Inicie o terraform 
 * terraform init

E, por fim, aplique o terraform
 * terraform apply -var-file=bentoctl.tfvars -auto-approve

# Conclusão

Com todos os passos seguidos corretamente, conseguimos disponibilizar a API em um endpoint público através do AWS Ec2, testando com um requisição post ao IP público da instância. Não vamos disponibilizar as infomações do endpoint pois desligamos para não gastar com a conta AWS. A t2.micro apesar de estar em free-tier tem uma limitação de tempo e não queremos correr o risco. Caso seja de interesse, podemos subir a aplicação novamente para mostrar o endpoint funcionando.  


Outro ponto interessante sobre o bentoml é que ele já possui uma integração fácil com o Prometheus através do endpoint /metrics/ que é criado automaticamente. Isso ajuda bastante num cenário onde é preciso monitorar o funcionamento do deploy.